In [94]:
from dask.distributed import Client
import pandas as pd
#!pip install s3fs
c = Client(address="18.130.202.146:8786")
#c.restart()
c

Client Scheduler: tcp://18.130.202.146:8786 Dashboard: http://18.130.202.146:8787/status,Cluster Workers: 50 Cores: 200 Memory: 399.86 GB


In [165]:
c.restart()

Client Scheduler: tcp://18.130.202.146:8786 Dashboard: http://18.130.202.146:8787/status,Cluster Workers: 50 Cores: 200 Memory: 399.86 GB


In [164]:
c.cancel(df_dask)

In [166]:
import dask.dataframe as dd
df_dask = dd.read_csv(["s3://nyc-tlc/trip data/yellow_tripdata_2015*.csv",
                       "s3://nyc-tlc/trip data/yellow_tripdata_2016*.csv",
                       "s3://nyc-tlc/trip data/yellow_tripdata_2017*.csv",
                       "s3://nyc-tlc/trip data/yellow_tripdata_2018*.csv"
                      ], dtype={'payment_type': 'float64', 'pickup_latitude': 'object', 'tpep_pickup_datetime': "datetime"}).persist()

TypeError: data type "datetime" not understood

In [161]:
df_dask['tpep_pickup_datetime'].head()


0    2015-01-15 19:05:39
1    2015-01-10 20:33:38
2    2015-01-10 20:33:38
3    2015-01-10 20:33:39
4    2015-01-10 20:33:39
Name: tpep_pickup_datetime, dtype: object

In [10]:
%%time
df_dask.describe().compute()

,VendorID,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
count,1.134969e+08,1.134969e+08,1.134969e+08,1.134969e+08,1.134969e+08,1.134969e+08,1.134969e+08,1.134969e+08,1.134969e+08,1.134969e+08,1.134969e+08,1.134969e+08,1.134969e+08,1.134969e+08
mean,1.549238e+00,1.625285e+00,2.928660e+00,1.044313e+00,1.629184e+02,1.608658e+02,1.336571e+00,1.304798e+01,3.312383e-01,4.972916e-01,1.837307e+00,3.252319e-01,2.996208e-01,1.634269e+01
std,4.975697e-01,1.265134e+00,4.073915e+00,5.824105e-01,6.656727e+01,7.049244e+01,4.930003e-01,1.903604e+02,1.391101e+00,6.906021e-02,2.624817e+00,1.946531e+00,1.430863e-02,1.905497e+02
min,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,-5.500000e+02,-5.864000e+01,-5.000000e-01,-3.910000e+02,-2.400000e+01,-3.000000e-01,-5.503000e+02
25%,1.000000e+00,1.000000e+00,1.090000e+00,1.000000e+00,1.320000e+02,1.140000e+02,1.000000e+00,7.000000e+00,0.000000e+00,5.000000e-01,0.000000e+00,0.000000e+00,3.000000e-01,9.360000e+00
50%,2.000000e+00,1.000000e+00,1.830000e+00,1.000000e+00,1.620000e+02,1.630000e+02,1.000000e+00,1.100000e+01,5.000000e-01,5.000000e-01,1.550000e+00,0.000000e+00,3.000000e-01,1.410000e+01
75%,2.000000e+00,2.000000e+00,3.510000e+00,1.000000e+00,2.340000e+02,2.340000e+02,2.000000e+00,1.850000e+01,1.000000e+00,5.000000e-01,2.960000e+00,0.000000e+00,3.000000e-01,2.230000e+01
max,2.000000e+00,1.920000e+02,9.496980e+03,9.900000e+01,2.650000e+02,2.650000e+02,5.000000e+00,8.616045e+05,1.400000e+04,2.500000e+02,1.600220e+03,5.743510e+03,1.000000e+00,8.616111e+05


In [97]:
df_dask['VendorID'].unique().compute()

0    1
1    2
2    4
Name: VendorID, dtype: int64

In [162]:
%%time
df_dask.groupby("passenger_count").aggregate({"total_amount": ["mean"]}).compute()


ValueError: Length mismatch: Expected axis has 17 elements, new values have 19 elements

In [157]:
print(df_dask['tpep_pickup_datetime'].head())
from pandas import to_datetime
meta = ('time', pd.to_datetime)
#df_dask['tpep_pickup_datetime'].map_partitions(to_datetime, meta=meta).persist()
df_dask['time'] = df_dask['tpep_pickup_datetime'].map(lambda x: pd.to_datetime(x, errors='coerce')).persist()
df_dask['trip_distance_int'] = df_dask['trip_distance'].map(lambda x: int(x)).persist()

0    2015-01-15 19:05:39
1    2015-01-10 20:33:38
2    2015-01-10 20:33:38
3    2015-01-10 20:33:39
4    2015-01-10 20:33:39
Name: tpep_pickup_datetime, dtype: object


In [158]:
df_dask['trip_distance_int'].head()

0    1
1    3
2    1
3    0
4    3
Name: trip_distance_int, dtype: int64

In [159]:
df_dask['time'].dt.year.head()

0    2015
1    2015
2    2015
3    2015
4    2015
Name: time, dtype: int64

In [163]:
%%time
df_dask.groupby([df_dask["passenger_count"], df_dask['time'].dt.year]).aggregate({"total_amount": ["count"]}).compute()


ValueError: Length mismatch: Expected axis has 17 elements, new values have 19 elements

In [150]:
%%time
df_dask.groupby([df_dask["passenger_count"], df_dask['time'].dt.year, df_dask["trip_distance_int"]]).aggregate({"total_amount": ["count"]}).compute().sort_values(["time", ("total_amount", "count")])

ValueError: Mismatched dtypes found in `pd.read_csv`/`pd.read_table`.

+-----------------+---------+----------+
| Column          | Found   | Expected |
+-----------------+---------+----------+
| payment_type    | float64 | int64    |
| pickup_latitude | object  | float64  |
+-----------------+---------+----------+

The following columns also raised exceptions on conversion:

- pickup_latitude
  ValueError("could not convert string to float: 'N'",)

Usually this is due to dask's dtype inference failing, and
*may* be fixed by specifying dtypes manually by adding:

dtype={'payment_type': 'float64',
       'pickup_latitude': 'object'}

to the call to `read_csv`/`read_table`.

In [151]:
agg = df_dask.groupby([df_dask["passenger_count"], df_dask['time'].dt.year, df_dask["trip_distance_int"]]).aggregate({"total_amount": ["count"]})

In [152]:
res.sort_values(["time", ("total_amount", "count")])

total_amount
                                              count
passenger_count time trip_distance_int             
1               2001 1                            1
                     4                            1
5               2001 1                            1
1               2001 0                            7
                2002 2                            1
                     3                            1
                     9                            1
                     11                           1
                     19                           1
2               2002 4                            1
                     7                            1
4               2002 1                            1
5               2002 0                            1
1               2002 0                            2
3               2002 1                            2
1               2002 1                            3
                2003 1                            1
                     3                            1
                     7                            1
                     10                           1
                     12                           1
2               2003 7                            1
6               2003 18                           1
1               2003 20                           2
                     0                            7
                2008 14                           1
                     15                           1
                     16                           1
                     20                           1
                     40                           1
...                                             ...
                2018 4                      1496390
2               2018 0                      1936368
                     1                      2634276
1               2018 3                      2725566
                     2                      5823176
                     0                     10160084
                     1                     13007802
                2019 4                            1
                     11                           1
2               2019 0                            2
                     1                            2
1               2019 2                            3
2               2019 2                            4
1               2019 0                            7
                     1                            9
                2020 2                            1
                     0                            2
                     1                            3
2               2026 0                            1
                     3                            1
1               2029 11                           1
4               2029 4                            1
1               2031 1                            2
                2037 2                            1
                2041 1                            1
                     8                            1
                2042 0                            1
3               2053 0                            1
1               2084 0                            2
                     1                            6

[1733 rows x 1 columns]

In [ ]:
%%time
df_dask.groupby("VendorID").aggregate({"trip_total":["sum","mean", "min"],
                                "tips":["sum","mean", "max"]})






In [ ]:
import pandas as pd
import glob 
files = glob.glob("s3://dask-fargate-data-naxty" +"/chicago_taxi_trips_2016_0*.csv")
files = [
    "s3://dask-fargate-data-naxty" +"/chicago_taxi_trips_2016_01.csv" ,
    "s3://dask-fargate-data-naxty" +"/chicago_taxi_trips_2016_02.csv", 
    "s3://dask-fargate-data-naxty" +"/chicago_taxi_trips_2016_03.csv", 
    "s3://dask-fargate-data-naxty" +"/chicago_taxi_trips_2016_04.csv", 
    "s3://dask-fargate-data-naxty" +"/chicago_taxi_trips_2016_05.csv", 
    "s3://dask-fargate-data-naxty" +"/chicago_taxi_trips_2016_06.csv", 
    "s3://dask-fargate-data-naxty" +"/chicago_taxi_trips_2016_07.csv", 
    "s3://dask-fargate-data-naxty" +"/chicago_taxi_trips_2016_08.csv", 
    "s3://dask-fargate-data-naxty" +"/chicago_taxi_trips_2016_09.csv", 
    "s3://dask-fargate-data-naxty" +"/chicago_taxi_trips_2016_10.csv", 
    "s3://dask-fargate-data-naxty" +"/chicago_taxi_trips_2016_11.csv", 
    "s3://dask-fargate-data-naxty" +"/chicago_taxi_trips_2016_12.csv", 
  
        ]
print(files)
df = pd.concat([pd.read_csv(file, dtype={'taxi_id': 'float64'}) for file in files])

In [ ]:
%%time
df.groupby("taxi_id").aggregate({
    "trip_distance":["sum","mean", "min", "max"],
    "tip_amount":["sum","mean", "min", "max"],
    "tolls_amount":["sum","mean", "min", "max"]
})

In [ ]:
!aws s3 ls nyc-tlc

In [ ]:
!pip install awscli

In [ ]:
x = dd.read_csv("s3://nyc-tlc/trip data/yellow_tripdata_2018-01.csv").compute()

In [ ]:
x

In [ ]:
x.shape

In [87]:
df_dask['tpep_pickup_datetime'].head()

TypeError: 'Future' object is not subscriptable

In [100]:
df_dask['tpep_pickup_datetime'].head()


CancelledError: ('from-delayed-623b69c403a550b9608d0710b7dbdae9', 0)

KeyboardInterrupt: 